# Cassava Augmentations

Below augmentation utility functions are copied from [ihelon](https://www.kaggle.com/ihelon) notebook, Thanks for the excellent resource [ihelon](https://www.kaggle.com/ihelon)

Please have a look at his work once.

https://www.kaggle.com/ihelon/monet-visualization-and-augmentation

In [ ]:
import os
import math
import random
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import albumentations as A
import torchvision
from PIL import Image
import torch
import numpy as np

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
    
SEED = 42
set_seed(SEED)


In [ ]:
BASE_PATH = '../input/cassava-leaf-disease-classification/'
IMAGE_PATH = os.path.join(BASE_PATH, "train_images")
IMAGE = '../input/cassava-leaf-disease-classification/train_images/469487.jpg'
IMAGE_SIZE_AUG = 256
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
p=0.5

In [ ]:
augmentations = A.Compose(
        [     
        A.RandomResizedCrop(IMAGE_SIZE_AUG, IMAGE_SIZE_AUG),
        A.Transpose(p=p),
        A.HorizontalFlip(p=p),
        A.VerticalFlip(p=p),
        A.OneOf([
        A.IAAAdditiveGaussianNoise(),
        A.GaussNoise(),
        ], p=p),
        A.ShiftScaleRotate(p=p),
        A.Normalize(mean, std, max_pixel_value=255.0, always_apply=True),  
        ]
    )

In [ ]:
class Cassava_Train_DS:
    
    def __init__(self, image_paths, targets, resize, augmentations=None):
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item])
        targets = self.targets[item]
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )
        image = np.array(image)
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        dct = {
            "image": torch.tensor(image),
            "targets": torch.tensor(targets),
        }
                
        return dct

In [ ]:
def _transform_aug(
    input_path='../input/cassava-leaf-disease-classification/',
    image_path = '../input/cassava-leaf-disease-classification/train_images/',
    resize = (256, 256)
):
           
    train_df = pd.read_csv(input_path+'train.csv')
    train_images = train_df.image_id.values.tolist()
    train_images = [os.path.join(image_path, i) for i in train_images]
    train_targets = train_df.label.values   
    train_dataset = Cassava_Train_DS(train_images, train_targets, resize = resize, augmentations = augmentations)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
    for idx, data in enumerate(trainloader): 
        break
    print(data['targets'].numpy())
    im = torchvision.utils.make_grid(data['image'], nrow=8)  # the default nrow is 8
    inv_normalize = torchvision.transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    im_inv = inv_normalize(im)
    plt.figure(figsize=(17,15))
    plt.imshow(np.transpose(im_inv.numpy(), (1, 2, 0)));

In [ ]:
#Batch Visualizer
_transform_aug()

In [ ]:
def batch_visualization(path, n_images, is_random=True, figsize=(16, 16)):
    plt.figure(figsize=figsize)
    
    w = int(n_images ** .5)
    h = math.ceil(n_images / w)
    all_names = os.listdir(path)

    image_names = all_names[:n_images]
    if is_random:
        image_names = random.sample(all_names, n_images)
    
    for ind, image_name in enumerate(image_names):
        img = cv2.imread(os.path.join(path, image_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
        plt.subplot(h, w, ind + 1)
        plt.imshow(img)
        plt.axis("off")
    
    plt.show()

In [ ]:
batch_visualization(IMAGE_PATH, 1, is_random=True, figsize=(10, 6))

In [ ]:
batch_visualization(IMAGE_PATH, 16, is_random=True, figsize=(10, 10))

In [ ]:
batch_visualization(IMAGE_PATH, 100, is_random=False)

In [ ]:
def color_hist_visualization(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 4, 1)
    plt.imshow(img)
    plt.axis("off")
    
    colors = ["red", "green", "blue"]
    for i in range(len(colors)):
        plt.subplot(1, 4, i + 2)
        plt.hist(
            img[:, :, i].reshape(-1),
            bins=25,
            alpha=0.5,
            color=colors[i],
            density=True
        )
        plt.xlim(0, 255)
        plt.xticks([])
        plt.yticks([])
    
    
    plt.show()

In [ ]:
color_hist_visualization(IMAGE)

color_hist_visualization(IMAGE)

color_hist_visualization(IMAGE)

In [ ]:
def channels_visualization(image_path, figsize=(16, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 4, 1)
    plt.imshow(np.mean(img, axis=2), cmap="gray")
    plt.axis('off')
    
    for i in range(3):
        plt.subplot(1, 4, i + 2)
        tmp_img = np.full_like(img, 0)
        tmp_img[:, :, i] = img[:, :, i]
        plt.imshow(tmp_img)
        plt.xlim(0, 255)
        plt.xticks([])
        plt.yticks([])
    
    
    plt.show()

In [ ]:
channels_visualization(IMAGE)

In [ ]:
def grayscale_visualization(image_path, figsize=(8, 4)):
    plt.figure(figsize=figsize)
    
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    tmp_img = np.full_like(img, 0)
    for i in range(3):
        tmp_img[:, :, i] = img.mean(axis=-1)
    plt.imshow(tmp_img)
    plt.axis('off')
    
    
    plt.show()

In [ ]:
grayscale_visualization(IMAGE)

In [ ]:
def plot_simple_augmentation(image_path, transform):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(1, 2, 2)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")

    plt.show()
    
def plot_multiple_augmentation(image_path, transform):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    
    plt.figure(figsize=(10, 10))
    
    plt.subplot(2, 2, 1)
    plt.imshow(img)
    plt.axis("off")
    
    plt.subplot(2, 2, 2)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")
    
    plt.subplot(2, 2, 3)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")
    
    plt.subplot(2, 2, 4)
    x = transform(image=img)["image"]
    plt.imshow(x)
    plt.axis("off")

    plt.show()

# Blur the input image using a random-sized kernel.

Default: albumentations.augmentations.transforms.Blur (blur_limit=7, always_apply=False, p=0.5)

In [ ]:
transform = A.Compose(
    [
        A.Blur(p=1.0, blur_limit=(5, 5)),
    ]
)

plot_simple_augmentation(IMAGE, transform)

# Apply Contrast Limited Adaptive Histogram Equalization to the input image.

Default: albumentations.augmentations.transforms.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5)

In [ ]:
transform = A.CLAHE(p=1.0, clip_limit=(10, 10), tile_grid_size=(3, 3))

plot_simple_augmentation(IMAGE, transform)

# Crop the central part of the input.

Default: albumentations.augmentations.transforms.CenterCrop (height, width, always_apply=False, p=1.0)

In [ ]:
transform = A.CenterCrop(p=1.0, height=100, width=150)

plot_simple_augmentation(IMAGE, transform)

# Randomly Drop Channels in the input Image.

Default: albumentations.augmentations.transforms.ChannelDropout (channel_drop_range=(1, 1), fill_value=0, always_apply=False, p=0.5)

In [ ]:
transform = A.ChannelDropout(p=1.0, channel_drop_range=(1, 2), fill_value=0)

plot_multiple_augmentation(IMAGE, transform)

# Randomly rearrange channels of the input RGB image.

Default: albumentations.augmentations.transforms.ChannelShuffle(p=0.5)

In [ ]:
transform = A.ChannelShuffle(p=1.0)

plot_multiple_augmentation(IMAGE, transform)

# Crop region from image.

Default: albumentations.augmentations.transforms.Crop (x_min=0, y_min=0, x_max=1024, y_max=1024, always_apply=False, p=1.0)

In [ ]:
transform = A.Crop(p=1.0, x_min=0, y_min=0, x_max=150, y_max=150)

plot_simple_augmentation(IMAGE, transform)

# Crop a random part of the input.

Default: albumentations.augmentations.transforms.RandomCrop (height, width, always_apply=False, p=1.0)

In [ ]:
transform = A.RandomCrop(p=1.0, height=100, width=100)

plot_multiple_augmentation(IMAGE, transform)

# CoarseDropout of the rectangular regions in the image.

Default: albumentations.augmentations.transforms.CoarseDropout (max_holes=8, max_height=8, max_width=8, min_holes=None, min_height=None, min_width=None, fill_value=0, mask_fill_value=None, always_apply=False, p=0.5)

In [ ]:
transform = A.CoarseDropout(
    max_holes=8,
    max_height=8,
    max_width=8, 
    min_holes=None,
    min_height=None,
    min_width=None, 
    fill_value=0, 
    always_apply=False,
    p=0.5
)

plot_multiple_augmentation(IMAGE, transform)


# Decreases image quality by downscaling and upscaling back.

Default: albumentations.augmentations.transforms.Downscale (scale_min=0.25, scale_max=0.25, interpolation=0, always_apply=False, p=0.5)

In [ ]:
transform = A.Downscale(
    p=1.0, scale_min=0.01, scale_max=0.20, interpolation=0,
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Equalize the image histogram.

Default: albumentations.augmentations.transforms.Equalize (mode='cv', by_channels=True, mask=None, mask_params=(), always_apply=False, p=0.5)

In [ ]:
transform = A.Equalize(
    p=1.0, mode='cv', by_channels=True,
)

plot_simple_augmentation(
    IMAGE,
    transform,
)

# Flip the input horizontally around the y-axis.

Default: albumentations.augmentations.transforms.HorizontalFlip(p=0.5)

In [ ]:
transform = A.HorizontalFlip(
    p=1,
)

plot_simple_augmentation(
    IMAGE,
    transform,
)

# Flip the input vertically around the x-axis.

Default: albumentations.augmentations.transforms.VerticalFlip(p=0.5)

In [ ]:
transform = A.VerticalFlip(
    p=1.0,
)

plot_simple_augmentation(
    IMAGE,
    transform,
)

# Flip the input either horizontally, vertically or both horizontally and vertically.

Default: albumentations.augmentations.transforms.Flip(p=0.5)

In [ ]:
transform = A.Flip(
    p=1.0,
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Apply gaussian noise to the input image.

Default: albumentations.augmentations.transforms.GaussNoise (var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5)

In [ ]:
transform = A.GaussNoise(
    p=1.0, var_limit=(500.0, 500.0),
)

plot_simple_augmentation(
    IMAGE,
    transform,
)

# Grid Distortion

Default: albumentations.augmentations.transforms.GridDistortion (num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5)

In [ ]:
transform = A.GridDistortion(
    p=1.0, num_steps=15, distort_limit=(-2., 2.), 
    interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None,
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Randomly change hue, saturation and value of the input image.

Default: albumentations.augmentations.transforms.HueSaturationValue (hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=0.5)

In [ ]:
transform = A.HueSaturationValue(
    p=1.0, 
    hue_shift_limit=(-100, 100), 
    sat_shift_limit=(-100, 100), 
    val_shift_limit=(-100, 100),
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Apply camera sensor noise.

Default: albumentations.augmentations.transforms.ISONoise (color_shift=(0.01, 0.05), intensity=(0.1, 0.5), always_apply=False, p=0.5)

In [ ]:
transform = A.ISONoise(
    p=1.0, intensity=(0.0, 2.0), color_shift=(0.0, 1.0)
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Decrease Jpeg, WebP compression of an image.

Default: albumentations.augmentations.transforms.ImageCompression (quality_lower=99, quality_upper=100, compression_type=<ImageCompressionType.JPEG: 0>, always_apply=False, p=0.5)

In [ ]:
transform = A.ImageCompression(
    p=1.0, quality_lower=0, quality_upper=10, compression_type=0,
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Invert the input image by subtracting pixel values from 255.

Default: albumentations.augmentations.transforms.InvertImg(p=0.5)

In [ ]:
transform = A.InvertImg(
    p=1.0,
)

plot_simple_augmentation(
    IMAGE,
    transform,
)

# Decrease Jpeg compression of an image.

Default: albumentations.augmentations.transforms.JpegCompression (quality_lower=99, quality_upper=100, always_apply=False, p=0.5)

In [ ]:
transform = A.JpegCompression(
    p=1.0, quality_lower=0, quality_upper=10,
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Apply motion blur to the input image using a random-sized kernel.

Default: albumentations.augmentations.transforms.MotionBlur(p=0.5)

In [ ]:
transform = A.MotionBlur(
    p=1.0, blur_limit=(3, 50),
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Multiply image to random number or array of numbers.

Default: albumentations.augmentations.transforms.MultiplicativeNoise (multiplier=(0.9, 1.1), per_channel=False, elementwise=False, always_apply=False, p=0.5)

In [ ]:
transform = A.MultiplicativeNoise(
    p=1.0, multiplier=(0.1, 5.0), per_channel=True, elementwise=False,
)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Randomly apply affine transforms: translate, scale and rotate the input.

Default: albumentations.augmentations.transforms.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, interpolation=1, border_mode=4, always_apply=False, p=0.5)

In [ ]:
transform = A.ShiftScaleRotate(p=1)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Apply gaussian noise to the input image.

Default: albumentations.augmentations.transforms.GaussNoise (var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5)

In [ ]:
transform = A.GaussNoise(p=1)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Randomly change hue, saturation and value of the input image.

Default: albumentations.augmentations.transforms.HueSaturationValue (hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=0.5)

In [ ]:
transform = A.HueSaturationValue (hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=1)

plot_multiple_augmentation(
    IMAGE,
    transform,
)

# Randomly change brightness of the input image.

Default: albumentations.augmentations.transforms.RandomBrightness (limit=0.2, always_apply=False, p=0.5)

In [ ]:
transform = A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, brightness_by_max=True, always_apply=False, p=1)
plot_multiple_augmentation(
    IMAGE,
    transform,
)